# KNN

In [1]:
# In this assignment, students will be using the K-nearest neighbors
# algorithm to predict how many points NBA players scored in the 2013-2014
# season.

In [2]:
import pandas as pd
with open("nba_2013.csv", 'r') as csvfile:
    nba = pd.read_csv(csvfile)

In [3]:
nba.columns

Index(['player', 'pos', 'age', 'bref_team_id', 'g', 'gs', 'mp', 'fg', 'fga',
       'fg.', 'x3p', 'x3pa', 'x3p.', 'x2p', 'x2pa', 'x2p.', 'efg.', 'ft',
       'fta', 'ft.', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf',
       'pts', 'season', 'season_end'],
      dtype='object')

In [4]:
nba.isnull().sum()

player           0
pos              0
age              0
bref_team_id     0
g                0
gs               0
mp               0
fg               0
fga              0
fg.              2
x3p              0
x3pa             0
x3p.            67
x2p              0
x2pa             0
x2p.             3
efg.             2
ft               0
fta              0
ft.             20
orb              0
drb              0
trb              0
ast              0
stl              0
blk              0
tov              0
pf               0
pts              0
season           0
season_end       0
dtype: int64

In [5]:
nba['fg.'].fillna(nba['fg.'].mean(), inplace=True)
nba["x2p."].fillna(nba["x2p."].mean(),inplace=True)
nba["efg."].fillna(nba["efg."].mean(),inplace=True)
nba["x3p."].fillna(nba["x3p."].mean(),inplace=True)
nba["ft."].fillna(nba["ft."].mean(),inplace=True)

In [6]:
distance_columns = ['age', 'g', 'gs', 'mp', 'fg', 'fga', 'fg.', 'x3p', 'x3pa', 'x3p.', 'x2p', 'x2pa', 'x2p.', 'efg.', 'ft', 'fta', 'ft.', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts']
nba_numeric = nba[distance_columns]

In [7]:
nba_normalized = nba_numeric.apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [8]:
nba_category = nba[['player', 'bref_team_id', 'season']]

In [9]:
nba

,player,pos,age,bref_team_id,g,gs,mp,fg,fga,fg.,...,drb,trb,ast,stl,blk,tov,pf,pts,season,season_end
0,Quincy Acy,SF,23,TOT,63,0,847,66,141,0.468,...,144,216,28,23,26,30,122,171,2013-2014,2013
1,Steven Adams,C,20,OKC,81,20,1197,93,185,0.503,...,190,332,43,40,57,71,203,265,2013-2014,2013
2,Jeff Adrien,PF,27,TOT,53,12,961,143,275,0.520,...,204,306,38,24,36,39,108,362,2013-2014,2013
3,Arron Afflalo,SG,28,ORL,73,73,2552,464,1011,0.459,...,230,262,248,35,3,146,136,1330,2013-2014,2013
4,Alexis Ajinca,C,25,NOP,56,30,951,136,249,0.546,...,183,277,40,23,46,63,187,328,2013-2014,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476,Tony Wroten,SG,20,PHI,72,16,1765,345,808,0.427,...,159,228,217,78,16,204,151,939,2013-2014,2013
477,Nick Young,SG,28,LAL,64,9,1810,387,889,0.435,...,137,166,95,46,12,95,156,1144,2013-2014,2013
478,Thaddeus Young,PF,25,PHI,79,78,2718,582,1283,0.454,...,310,476,182,167,36,165,213,1417,2013-2014,2013
479,Cody Zeller,C,21,CHA,82,3,1416,172,404,0.426,...,235,353,92,40,41,87,170,490,2013-2014,2013


In [10]:
nba_numeric

,age,g,gs,mp,fg,fga,fg.,x3p,x3pa,x3p.,...,ft.,orb,drb,trb,ast,stl,blk,tov,pf,pts
0,23,63,0,847,66,141,0.468,4,15,0.266667,...,0.660,72,144,216,28,23,26,30,122,171
1,20,81,20,1197,93,185,0.503,0,0,0.285111,...,0.581,142,190,332,43,40,57,71,203,265
2,27,53,12,961,143,275,0.520,0,0,0.285111,...,0.639,102,204,306,38,24,36,39,108,362
3,28,73,73,2552,464,1011,0.459,128,300,0.426667,...,0.815,32,230,262,248,35,3,146,136,1330
4,25,56,30,951,136,249,0.546,0,1,0.000000,...,0.836,94,183,277,40,23,46,63,187,328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476,20,72,16,1765,345,808,0.427,40,188,0.212766,...,0.641,69,159,228,217,78,16,204,151,939
477,28,64,9,1810,387,889,0.435,135,350,0.385714,...,0.825,29,137,166,95,46,12,95,156,1144
478,25,79,78,2718,582,1283,0.454,90,292,0.308219,...,0.712,166,310,476,182,167,36,165,213,1417
479,21,82,3,1416,172,404,0.426,0,1,0.000000,...,0.730,118,235,353,92,40,41,87,170,490


In [11]:
nba_category

,player,bref_team_id,season
0,Quincy Acy,TOT,2013-2014
1,Steven Adams,OKC,2013-2014
2,Jeff Adrien,TOT,2013-2014
3,Arron Afflalo,ORL,2013-2014
4,Alexis Ajinca,NOP,2013-2014
...,...,...,...
476,Tony Wroten,PHI,2013-2014
477,Nick Young,LAL,2013-2014
478,Thaddeus Young,PHI,2013-2014
479,Cody Zeller,CHA,2013-2014


In [12]:
nba = pd.concat([nba_category, nba_normalized], axis=1)
nba

,player,bref_team_id,season,age,g,gs,mp,fg,fga,fg.,...,ft.,orb,drb,trb,ast,stl,blk,tov,pf,pts
0,Quincy Acy,TOT,2013-2014,0.20,0.756098,0.000000,0.271067,0.077739,0.083531,0.468,...,0.660,0.163636,0.183908,0.193896,0.038835,0.120419,0.118721,0.101695,0.446886,0.065947
1,Steven Adams,OKC,2013-2014,0.05,0.975610,0.243902,0.383211,0.109541,0.109597,0.503,...,0.581,0.322727,0.242656,0.298025,0.059639,0.209424,0.260274,0.240678,0.743590,0.102198
2,Jeff Adrien,TOT,2013-2014,0.40,0.634146,0.146341,0.307594,0.168433,0.162915,0.520,...,0.639,0.231818,0.260536,0.274686,0.052705,0.125654,0.164384,0.132203,0.395604,0.139607
3,Arron Afflalo,ORL,2013-2014,0.45,0.878049,0.890244,0.817366,0.546525,0.598934,0.459,...,0.815,0.072727,0.293742,0.235189,0.343967,0.183246,0.013699,0.494915,0.498168,0.512919
4,Alexis Ajinca,NOP,2013-2014,0.30,0.670732,0.365854,0.304390,0.160188,0.147512,0.546,...,0.836,0.213636,0.233716,0.248654,0.055479,0.120419,0.210046,0.213559,0.684982,0.126494
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476,Tony Wroten,PHI,2013-2014,0.05,0.865854,0.195122,0.565203,0.406360,0.478673,0.427,...,0.641,0.156818,0.203065,0.204668,0.300971,0.408377,0.073059,0.691525,0.553114,0.362129
477,Nick Young,LAL,2013-2014,0.45,0.768293,0.109756,0.579622,0.455830,0.526659,0.435,...,0.825,0.065909,0.174968,0.149013,0.131761,0.240838,0.054795,0.322034,0.571429,0.441188
478,Thaddeus Young,PHI,2013-2014,0.30,0.951220,0.951220,0.870554,0.685512,0.760071,0.454,...,0.712,0.377273,0.395913,0.427289,0.252427,0.874346,0.164384,0.559322,0.780220,0.546471
479,Cody Zeller,CHA,2013-2014,0.10,0.987805,0.036585,0.453380,0.202591,0.239336,0.426,...,0.730,0.268182,0.300128,0.316876,0.127601,0.209424,0.187215,0.294915,0.622711,0.188970


In [13]:
from sklearn.model_selection import train_test_split
x = nba[['age', 'g', 'gs', 'mp', 'fg', 'fga', 'fg.', 'x3p', 'x3pa', 'x3p.', 'x2p', 'x2pa', 'x2p.', 'efg.', 'ft', 'fta', 'ft.', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf']]
y = nba["pts"]

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

In [15]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn import metrics


knn = KNeighborsRegressor()
knn.fit(x_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')

In [16]:
y_pred = knn.predict(x_test)

In [17]:
knn.score(x_train,y_train)

0.9750977183338924

In [18]:
knn.score(x_test, y_test)

0.9582816514945678

In [19]:
for k in range(10):
    k_value = k + 1
    knn = KNeighborsRegressor(n_neighbors = k_value)
    knn.fit(x_train, y_train) 
    y_pred = knn.predict(x_test)
    print ("Score is:",format(metrics.r2_score(y_test, y_pred),'.4f'), "for k:", k_value)

Score is: 0.9145 for k: 1
Score is: 0.9464 for k: 2
Score is: 0.9548 for k: 3
Score is: 0.9594 for k: 4
Score is: 0.9583 for k: 5
Score is: 0.9579 for k: 6
Score is: 0.9579 for k: 7
Score is: 0.9609 for k: 8
Score is: 0.9576 for k: 9
Score is: 0.9557 for k: 10


In [20]:
knn = KNeighborsRegressor(n_neighbors = 8)
knn.fit(x_train, y_train) 
y_pred = knn.predict(x_test)
print ("Mean Squared Error is:", format(metrics.mean_squared_error(y_test, y_pred), '.7f'))
print ("Regression score is:", format(metrics.r2_score(y_test, y_pred),'.4f'))

Mean Squared Error is: 0.0011143
Regression score is: 0.9609


In [21]:
Test_With_Predicted = pd.DataFrame({'Actual Points': y_test.tolist(), 'Predicted Points': y_pred.tolist()})
Test_With_Predicted

,Actual Points,Predicted Points
0,0.168145,0.125723
1,0.276514,0.297243
2,0.422676,0.363189
3,0.007327,0.011088
4,0.381026,0.373939
...,...,...
140,0.426919,0.421664
141,0.013498,0.019379
142,0.312379,0.303943
143,0.306980,0.273766


In [22]:
import pickle
filename = 'Knn_assignment.pickle'
pickle.dump(knn, open(filename,'wb'))

In [23]:
loaded_model = pickle.load(open(filename,'rb'))

In [24]:
nba.iloc[1].values.tolist()

['Steven Adams',
 'OKC',
 '2013-2014',
 0.05,
 0.975609756097561,
 0.24390243902439024,
 0.38321050945209867,
 0.10954063604240283,
 0.10959715639810426,
 0.503,
 0.0,
 0.0,
 0.2851111897859946,
 0.13172804532577903,
 0.13139204545454544,
 0.502702702702703,
 0.503,
 0.112375533428165,
 0.168944099378882,
 0.581,
 0.32272727272727275,
 0.24265644955300128,
 0.2980251346499102,
 0.059639389736477116,
 0.2094240837696335,
 0.2602739726027397,
 0.24067796610169492,
 0.7435897435897436,
 0.10219822599305824]

In [25]:
#Just passing the same input params from input dataset direct to the model
#And also compare whether the original predicted value and the value predicted by my model is same or nearly same 

In [26]:
indexOfDataToPassToModel = 4
a = x.iloc[indexOfDataToPassToModel].values.tolist()
b = loaded_model.predict([a])
print ("original dataset pred val", nba['pts'][indexOfDataToPassToModel])
print ("predicted dataset pred val", b)

original dataset pred val 0.12649440802159662
predicted dataset pred val [0.14640378]
